In [1]:
import numpy as np
import pandas as pd

def worked_fitness_example():
    """
    Worked example of fitness function calculation
    Using a simplified scenario with 3 nodes, 2 energy sources, and 1 time slot
    """
    print("="*60)
    print("WORKED EXAMPLE: FITNESS FUNCTION CALCULATION")
    print("="*60)

    # Example data for demonstration
    print("STEP 1: Input Data")
    print("-" * 30)

    # Supply data (kWh available from each source)
    supply = np.array([500, 300])  # Solar: 500 kWh, Wind: 300 kWh
    source_names = ['Solar', 'Wind']
    print(f"Energy Supply: {dict(zip(source_names, supply))} kWh")

    # Demand data (kWh required at each node)
    demand = np.array([200, 250, 150])  # Node_0: 200, Node_1: 250, Node_2: 150
    node_names = ['Node_0', 'Node_1', 'Node_2']
    print(f"Energy Demand: {dict(zip(node_names, demand))} kWh")

    # Cost per kWh for each source
    costs = np.array([0.05, 0.08])  # Solar: $0.05/kWh, Wind: $0.08/kWh
    print(f"Energy Costs: {dict(zip(source_names, costs))} $/kWh")

    # Transmission loss rates (source to node)
    loss_matrix = np.array([
        [0.02, 0.05, 0.03],  # Solar to Node_0, Node_1, Node_2
        [0.04, 0.03, 0.06]   # Wind to Node_0, Node_1, Node_2
    ])
    print("Transmission Loss Matrix:")
    loss_df = pd.DataFrame(loss_matrix, index=source_names, columns=node_names)
    print(loss_df)

    # Example chromosome (allocation fractions)
    print("\nSTEP 2: Chromosome Representation")
    print("-" * 30)

    # Allocation matrix: allocation[i,j] = fraction of node j's demand met by source i
    allocation_matrix = np.array([
        [0.6, 0.4, 0.8],  # Solar allocation to each node
        [0.4, 0.6, 0.2]   # Wind allocation to each node
    ])

    print("Allocation Matrix (fractions):")
    alloc_df = pd.DataFrame(allocation_matrix, index=source_names, columns=node_names)
    print(alloc_df)

    print("\nSTEP 3: Fitness Calculation")
    print("-" * 30)

    # Initialize metrics
    total_cost = 0
    total_loss = 0
    capacity_penalty = 0
    demand_penalty = 0

    print("\nDetailed Calculations:")
    print("Node\tSource\tDemand\tAlloc%\tEnergy\tCost\tLoss\tNet Energy")
    print("-" * 70)

    # Track supply usage for capacity constraints
    supply_used = np.zeros(len(supply))

    for j in range(len(demand)):  # For each node
        node_demand = demand[j]
        total_supplied_to_node = 0

        for i in range(len(supply)):  # For each source
            # Calculate energy allocation
            allocation_fraction = allocation_matrix[i, j]
            energy_allocated = allocation_fraction * node_demand

            # Update supply usage
            supply_used[i] += energy_allocated

            # Check capacity constraints
            if supply_used[i] > supply[i]:
                excess = supply_used[i] - supply[i]
                capacity_penalty += excess ** 2
                energy_allocated = max(0, energy_allocated - excess)  # Reduce allocation

            # Calculate cost
            energy_cost = energy_allocated * costs[i]
            total_cost += energy_cost

            # Calculate transmission loss
            loss_rate = loss_matrix[i, j]
            energy_lost = energy_allocated * loss_rate
            total_loss += energy_lost

            # Net energy delivered to node
            net_energy = energy_allocated - energy_lost
            total_supplied_to_node += net_energy

            print(f"{node_names[j]}\t{source_names[i]}\t{node_demand:.0f}\t{allocation_fraction:.1f}\t"
                  f"{energy_allocated:.1f}\t${energy_cost:.2f}\t{energy_lost:.2f}\t{net_energy:.1f}")

        # Check demand fulfillment
        demand_deficit = max(0, node_demand - total_supplied_to_node)
        demand_penalty += demand_deficit ** 2

        if demand_deficit > 0:
            print(f"\t\tDemand Deficit for {node_names[j]}: {demand_deficit:.2f} kWh")

    # Calculate weighted fitness
    print("\nSTEP 4: Multi-Objective Fitness Computation")
    print("-" * 30)

    print(f"1. Total Cost: ${total_cost:.2f}")
    print(f"2. Total Transmission Loss: {total_loss:.2f} kWh")
    print(f"3. Capacity Penalty: {capacity_penalty:.2f}")
    print(f"4. Demand Penalty: {demand_penalty:.2f}")

    # Supply utilization analysis
    print(f"\nSupply Utilization:")
    for i, source in enumerate(source_names):
        utilization = (supply_used[i] / supply[i]) * 100
        print(f"{source}: {supply_used[i]:.1f}/{supply[i]:.1f} kWh ({utilization:.1f}%)")

    # Multi-objective fitness weights
    w1, w2, w3, w4 = 0.4, 0.3, 0.2, 0.1  # Cost, Loss, Demand, Capacity

    # Normalize penalties (scale up losses for proper weighting)
    scaled_loss = total_loss * 1000  # Convert kWh loss to equivalent cost scale

    # Calculate weighted fitness (to be minimized)
    fitness_components = {
        'cost': w1 * total_cost,
        'loss': w2 * scaled_loss,
        'demand_penalty': w3 * demand_penalty,
        'capacity_penalty': w4 * capacity_penalty
    }

    raw_fitness = sum(fitness_components.values())

    print(f"\nFitness Components:")
    for component, value in fitness_components.items():
        print(f"  {component}: {value:.2f} (weight: {[w1,w2,w3,w4][[k for k in fitness_components.keys()].index(component)]})")

    print(f"\nRaw Fitness (to minimize): {raw_fitness:.2f}")
    print(f"GA Fitness (to maximize): {-raw_fitness:.2f}")

    # Performance metrics
    total_demand_sum = np.sum(demand)
    total_supply_available = np.sum(supply)
    demand_fulfillment = ((total_demand_sum - np.sqrt(demand_penalty)) / total_demand_sum) * 100
    efficiency = ((total_supply_available - total_loss) / total_supply_available) * 100

    print(f"\nPerformance Metrics:")
    print(f"  Demand Fulfillment: {demand_fulfillment:.1f}%")
    print(f"  System Efficiency: {efficiency:.1f}%")
    print(f"  Average Cost per kWh: ${total_cost/total_demand_sum:.3f}")

    return {
        'total_cost': total_cost,
        'total_loss': total_loss,
        'capacity_penalty': capacity_penalty,
        'demand_penalty': demand_penalty,
        'fitness': -raw_fitness,
        'allocation_matrix': allocation_matrix
    }

def demonstrate_chromosome_encoding():
    """
    Demonstrate how chromosomes are encoded for the genetic algorithm
    """
    print("\n" + "="*60)
    print("CHROMOSOME ENCODING DEMONSTRATION")
    print("="*60)

    # Problem dimensions
    n_sources = 3  # Solar, Wind, Hydro
    n_nodes = 5    # 5 distribution nodes
    n_hours = 24   # 24-hour cycle

    print(f"Problem Dimensions:")
    print(f"  Sources: {n_sources}")
    print(f"  Nodes: {n_nodes}")
    print(f"  Time slots: {n_hours}")
    print(f"  Total genes: {n_sources * n_nodes * n_hours}")

    # Create sample chromosome
    np.random.seed(42)
    chromosome_3d = np.random.random((n_sources, n_nodes, n_hours))

    # Normalize for energy balance
    for h in range(n_hours):
        for j in range(n_nodes):
            total_allocation = chromosome_3d[:, j, h].sum()
            if total_allocation > 0:
                chromosome_3d[:, j, h] /= total_allocation

    chromosome_flat = chromosome_3d.flatten()

    print(f"\nChromosome Structure:")
    print(f"  3D Matrix Shape: {chromosome_3d.shape}")
    print(f"  Flattened Length: {len(chromosome_flat)}")

    # Show sample allocation for first hour
    print(f"\nSample Allocation (Hour 0):")
    source_names = ['Solar', 'Wind', 'Hydro']
    node_names = [f'Node_{i}' for i in range(n_nodes)]

    hour_0_df = pd.DataFrame(
        chromosome_3d[:, :, 0],
        index=source_names,
        columns=node_names
    )
    print(hour_0_df.round(3))

    # Verify normalization
    print(f"\nColumn sums (should be ~1.0):")
    print(hour_0_df.sum(axis=0).round(3).to_dict())

    return chromosome_flat

if __name__ == "__main__":
    # Run worked fitness example
    result = worked_fitness_example()

    # Demonstrate chromosome encoding
    sample_chromosome = demonstrate_chromosome_encoding()

    print(f"\n" + "="*60)
    print("EXAMPLE COMPLETE")
    print("="*60)

WORKED EXAMPLE: FITNESS FUNCTION CALCULATION
STEP 1: Input Data
------------------------------
Energy Supply: {'Solar': np.int64(500), 'Wind': np.int64(300)} kWh
Energy Demand: {'Node_0': np.int64(200), 'Node_1': np.int64(250), 'Node_2': np.int64(150)} kWh
Energy Costs: {'Solar': np.float64(0.05), 'Wind': np.float64(0.08)} $/kWh
Transmission Loss Matrix:
       Node_0  Node_1  Node_2
Solar    0.02    0.05    0.03
Wind     0.04    0.03    0.06

STEP 2: Chromosome Representation
------------------------------
Allocation Matrix (fractions):
       Node_0  Node_1  Node_2
Solar     0.6     0.4     0.8
Wind      0.4     0.6     0.2

STEP 3: Fitness Calculation
------------------------------

Detailed Calculations:
Node	Source	Demand	Alloc%	Energy	Cost	Loss	Net Energy
----------------------------------------------------------------------
Node_0	Solar	200	0.6	120.0	$6.00	2.40	117.6
Node_0	Wind	200	0.4	80.0	$6.40	3.20	76.8
		Demand Deficit for Node_0: 5.60 kWh
Node_1	Solar	250	0.4	100.0	$5.00	5